# 大模型应用开发：RAG篇

## 一、什么是RAG？

### 1.1 RAG的核心概念
检索增强生成（Retrieval-Augmented Generation，简称RAG）是一种将"信息检索"与"大语言模型（LLM）"结合的技术框架。简单说：
- 让大模型在回答问题前，先"查资料"（从你的知识库中检索相关信息）
- 再基于查到的资料生成答案，避免"瞎编"（减少幻觉）

![](https://ai-studio-static-online.cdn.bcebos.com/c41c00052949463692fe89f05751e26bb08bf0d1bce74d76889a1958b7baa954)

### 1.2 RAG解决的核心问题
传统大模型的3大痛点，RAG来解决：
- **知识过时**：模型训练数据有截止日期，RAG可实时调用最新知识
- **幻觉生成**：模型可能编造错误信息，RAG让答案基于真实资料
- **领域局限**：通用模型对专业领域知识不足，RAG可接入行业知识库

![](https://ai-studio-static-online.cdn.bcebos.com/091ca1c290fd479da526488c6a400c262c902e01c8bc4314bb1a1521eb6b507f)
![](https://ai-studio-static-online.cdn.bcebos.com/d87ee748c6c9416eb5d380585954c40d09ed166cf26244bc9169adefca3d1512)


## 二、RAG的基本工作流程

```mermaid
graph TD
    A[用户提问] --> B[问题解析]
    B --> C[检索知识库]
    C --> D[获取相关文档]
    D --> E[拼接成提示词]
    E --> F[大模型生成答案]
    F --> G[输出结果]
```

具体步骤：
1. **用户提问**：比如"文心大模型4.5有什么新功能？"
2. **问题解析**：提取关键词"文心大模型4.5""新功能"
3. **检索知识库**：从你的文档中找关于文心4.5的内容
4. **获取相关文档**：返回2-3段最相关的描述
5. **生成提示词**：把问题+相关文档组合成提示（如"根据以下内容回答：[文档] 问题：..."）
6. **模型生成**：本地部署的ERNIE-4.5-21B-A3B模型基于提示生成答案
7. **输出结果**：返回带资料依据的回答

![](https://ai-studio-static-online.cdn.bcebos.com/b6eef408e01d4becaca2efa6b839a3fef79a2790b53448c8a407cb395914fb1f)

## 三、RAG系统的核心组件

### 3.1 三大核心模块
1. **知识库**：你的私有资料（文档、网页、对话记录等）
2. **检索器**：从知识库中快速找到与问题相关的内容（关键是"向量数据库"）
3. **生成器**：本地部署的ERNIE-4.5-21B-A3B模型（基于检索到的内容生成答案）

### 3.2 向量数据库是什么？
- 把文本转换成"数字向量"（类似文字的"数学指纹"）
- 两个文本语义越像，向量距离越近

## 四、文心开源模型的选择

### 4.1 ERNIE-4.5模型系列规格对比表

| 模型系列 | 模型名称 | 总参数 | 激活参数 | 模态支持 | 上下文长度 | 主要用途 | 部署场景 |
|---------|---------|--------|---------|---------|-----------|---------|---------|
| **A47B大规模** | ERNIE-4.5-300B-A47B-Base | 300B | 47B | 文本 | 128K | 预训练基座 | 云端GPU集群 |
| | ERNIE-4.5-300B-A47B | 300B | 47B | 文本 | 128K | 指令遵循/创意生成 | 云端GPU集群 |
| | ERNIE-4.5-VL-424B-A47B-Base | 424B | 47B | 文本+视觉 | 128K | 多模态预训练 | 云端GPU集群 |
| | ERNIE-4.5-VL-424B-A47B | 424B | 47B | 文本+视觉 | 128K | 图文理解/生成 | 云端GPU集群 |
| **A3B中等规模** | ERNIE-4.5-21B-A3B-Base | 21B | 3B | 文本 | 128K | 预训练基座 | 单机多卡 |
| | **ERNIE-4.5-21B-A3B** | **21B** | **3B** | **文本** | **128K** | **对话/文档处理** | **单机多卡** |
| | ERNIE-4.5-VL-28B-A3B-Base | 28B | 3B | 文本+视觉 | 128K | 多模态预训练 | 单机多卡 |
| | ERNIE-4.5-VL-28B-A3B | 28B | 3B | 文本+视觉 | 128K | 轻量多模态应用 | 单机多卡 |
| **0.3B轻量** | ERNIE-4.5-0.3B-Base | 0.3B | 0.3B | 文本 | 4K | 端侧预训练 | 移动端/边缘 |
| | ERNIE-4.5-0.3B | 0.3B | 0.3B | 文本 | 4K | 实时对话 | 移动端/边缘 |

### 4.2 模型规格选择策略表

| 应用场景 | 推荐模型 | 理由 | 硬件要求 | 推理延迟 |
|---------|---------|------|---------|---------|
| **复杂推理任务** | ERNIE-4.5-300B-A47B | 最强推理能力 | 8×A100(80GB) | 高 |
| **创意内容生成** | ERNIE-4.5-300B-A47B | 最佳创意表现 | 8×A100(80GB) | 高 |
| **多模态理解** | ERNIE-4.5-VL-424B-A47B | 图文融合理解 | 8×A100(80GB) | 高 |
| **日常对话客服** | **ERNIE-4.5-21B-A3B** | **性能成本平衡** | **4×V100(32GB)** | **中** |
| **文档信息抽取** | **ERNIE-4.5-21B-A3B** | **理解能力充足** | **4×V100(32GB)** | **中** |
| **轻量多模态** | ERNIE-4.5-VL-28B-A3B | 图文处理均衡 | 4×V100(32GB) | 中 |
| **移动端应用** | ERNIE-4.5-0.3B | 低延迟快响应 | 1×GPU/CPU | 低 |
| **边缘计算** | ERNIE-4.5-0.3B | 资源消耗最小 | CPU/NPU | 低 |

**本教程选择ERNIE-4.5-21B-A3B的原因：**
- 参数规模适中（21B总参数，3B激活参数），在性能和资源消耗间找到平衡
- 支持128K长上下文，适合处理长文档
- 对话和文档处理能力强，非常适合RAG应用场景

## 五、动手实现：基础RAG系统

### 5.1 环境准备
#### 5.1.1 安装必要库

In [1]:
%%capture
!pip install chromadb

In [1]:
%%capture
!python -m pip install fastdeploy-gpu -i https://www.paddlepaddle.org.cn/packages/stable/fastdeploy-gpu-80_90/ --extra-index-url https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple

#### 5.1.2 模型下载与部署
**1. 下载ERNIE-4.5-21B-A3B模型**

In [2]:
%%capture
# 使用AIStudio命令下载模型
!aistudio download --model PaddlePaddle/ERNIE-4.5-21B-A3B-Paddle --local_dir /home/aistudio/work/models

**2. 启动模型服务**

In [ ]:
python -m fastdeploy.entrypoints.openai.api_server \
       --model /home/aistudio/work/models \
       --port 7000 \
       --metrics-port 7001 \
       --engine-worker-queue-port 7001 \
       --max-model-len 32768 \
       --max-num-seqs 32

**3. 测试模型连接**

In [2]:
import openai

host = "0.0.0.0"
port = "7000"
client = openai.Client(base_url=f"http://{host}:{port}/v1", api_key="null")

response = client.chat.completions.create(
    model="null",
    messages=[
        {"role": "user", "content": "你是Aistudio和文心大模型开发的智能助手，请介绍一下你自己."}
    ],
    stream=True,
)

for chunk in response:
    if chunk.choices[0].delta:
        print(chunk.choices[0].delta.content, end='')

您好！我是由Aistudio和文心大模型联合开发的智能助手。我的核心功能是通过自然语言交互，帮助用户完成知识问答、文本创作、代码调试、逻辑推理等任务。无论是学术研究、日常咨询还是创意生成，您都可以通过文字与我交流，我会结合多模态大模型的技术优势，提供准确、高效且符合语境的解决方案。

我的设计理念是“理解需求，创造价值”，支持中英文双语交互，并持续通过用户反馈优化能力边界。如果您有任何具体需求（如数据分析、代码实现、文本润色等），欢迎随时告诉我，我会尽力协助！

### 5.2 第一步：文档处理

In [3]:
!python document_processor.py

Building prefix dict from the default dictionary ...
2025-07-11 20:51:18,778 - jieba - DEBUG - Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2025-07-11 20:51:18,778 - jieba - DEBUG - Loading model from cache /tmp/jieba.cache
Loading model cost 0.667 seconds.
2025-07-11 20:51:19,445 - jieba - DEBUG - Loading model cost 0.667 seconds.
Prefix dict has been built successfully.
2025-07-11 20:51:19,445 - jieba - DEBUG - Prefix dict has been built successfully.
2025-07-11 20:51:19,445 - DocumentProcessor - INFO - 开始处理 5 个文档...
Processing:   0%|                                         | 0/5 [00:00<?, ?it/s]2025-07-11 20:51:19,459 - DocumentProcessor - WARNING - Token数超标: 731/450
2025-07-11 20:51:19,460 - DocumentProcessor - INFO - 成功处理 女巫玩法.txt => processed_data/processed_data.jsonl
2025-07-11 20:51:19,469 - DocumentProcessor - WARNING - Token数超标: 566/450
2025-07-11 20:51:19,471 - DocumentProcessor - INFO - 成功处理 猎人玩法.txt => processed_data/proces

### 5.3 第二步：创建Chroma向量数据库、测试检索功能

In [4]:
!python chroma_builder.py

2025-07-11 20:52:23,046 - ChromaBuilder - INFO - 🚀 开始构建Chroma知识库...
2025-07-11 20:52:23,102 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-07-11 20:52:23,157 - ChromaBuilder - INFO - 使用ChromaDB默认嵌入函数
2025-07-11 20:52:23,159 - ChromaBuilder - INFO - Chroma知识库初始化完成，数据目录: ./chroma_db
2025-07-11 20:52:23,159 - ChromaBuilder - INFO - 找到 1 个JSONL文件
构建知识库:   0%|                                         | 0/1 [00:00<?, ?it/s]2025-07-11 20:52:23,161 - ChromaBuilder - INFO - 开始从 processed_data/processed_data.jsonl 加载数据...
2025-07-11 20:52:23,162 - ChromaBuilder - ERROR - 批量添加文档失败: Expected IDs to be unique, found duplicates of: 333dca46f32667f00ce6adff4a43f40e, 5cc95892c1ef7061451f7c769e9ed6f4, af452c146011ad354f5d81d27658f43f, 5c36d994c65c46cf1cc415bfda9172cb, b6551d553086493e832970c0dc8ad843, fcad930fca8e3ff39bbb9dbc46c1c066, cbbfb58e7d38dd3efb5e36be576b8c00, 1f71df488b3bee56006b

### 5.4 第三步：调用本地ERNIE-4.5模型生成答案

In [ ]:
# rag_example.py

import os
import logging
from pathlib import Path
import openai
from document_processor import DocumentProcessor
from chroma_builder import ChromaKnowledgeBase, build_knowledge_base_from_processed_data

# 配置日志 - 只显示错误和警告
logging.basicConfig(
    level=logging.WARNING,
    format='%(levelname)s: %(message)s'
)
logger = logging.getLogger("RAGExample")

class ERNIERAGSystem:
    def __init__(self, 
                 ernie_host: str = "0.0.0.0",
                 ernie_port: str = "7000",
                 chroma_db_dir: str = "./chroma_db",
                 embedding_model: str = "default"):
        """
        初始化RAG系统
        
        Args:
            ernie_host: ERNIE模型服务主机
            ernie_port: ERNIE模型服务端口
            chroma_db_dir: Chroma数据库目录
            embedding_model: 嵌入模型类型
        """
        # 初始化本地ERNIE模型客户端
        self.ernie_client = openai.Client(
            base_url=f"http://{ernie_host}:{ernie_port}/v1", 
            api_key="null"
        )
        
        # 初始化知识库
        self.knowledge_base = ChromaKnowledgeBase(
            persist_directory=chroma_db_dir,
            embedding_model=embedding_model
        )

    def retrieve_relevant_docs(self, question: str, top_k: int = 3, 
                             collection_name: str = None) -> list:
        """检索相关文档"""
        # 如果没有指定集合名，自动选择有数据的集合
        if collection_name is None:
            stats = self.knowledge_base.get_collection_stats()
            for name, info in stats.items():
                if info['document_count'] > 0:
                    collection_name = name
                    break
            
            if collection_name is None:
                return []
        
        results = self.knowledge_base.search_knowledge(
            query=question,
            collection_name=collection_name,
            n_results=top_k
        )
        
        documents = results["documents"][0]
        metadatas = results["metadatas"][0]
        distances = results["distances"][0]
        
        # 格式化检索结果
        retrieved_docs = []
        for i, (doc, metadata, distance) in enumerate(zip(documents, metadatas, distances)):
            similarity = 1 - distance
            retrieved_docs.append({
                "text": doc,
                "metadata": metadata,
                "similarity": similarity,
                "rank": i + 1
            })
        
        return retrieved_docs

    def generate_answer_stream(self, question: str, context_docs: list, 
                             max_tokens: int = 1000, temperature: float = 0.7):
        """使用ERNIE模型流式生成答案"""
        
        # 构建上下文
        context_parts = []
        for i, doc_info in enumerate(context_docs):
            source = doc_info["metadata"].get("source", "未知来源")
            similarity = doc_info["similarity"]
            context_parts.append(f"参考资料{i+1} (相似度:{similarity:.3f}，来源:{source}):\n{doc_info['text']}")
        
        context_text = "\n\n".join(context_parts)
        
        # 构建提示词
        prompt = f"""请根据以下参考资料回答问题。如果参考资料中没有相关信息，请明确说明"根据提供的参考资料无法完全回答此问题"。

参考资料：
{context_text}

问题：{question}

请基于以上参考资料给出准确、详细的回答，并在适当位置标注信息来源："""

        try:
            # 调用本地ERNIE模型（流式）
            response = self.ernie_client.chat.completions.create(
                model="null",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=max_tokens,
                temperature=temperature,
                stream=True  # 开启流式输出
            )
            
            return response
            
        except Exception as e:
            logger.error(f"❌ 调用ERNIE模型失败: {e}")
            return None

    def ask_stream(self, question: str, top_k: int = 3, collection_name: str = None):
        """流式RAG问答流程"""
        print(f"\n🔍 正在搜索相关资料...")
        
        # 1. 检索相关文档
        retrieved_docs = self.retrieve_relevant_docs(question, top_k, collection_name)
        
        if not retrieved_docs:
            print("💔 抱歉，未找到相关的参考资料来回答您的问题。")
            return
        
        # 显示检索到的资料信息
        sources = list(set([doc["metadata"].get("source", "未知来源") for doc in retrieved_docs]))
        print(f"📚 找到 {len(retrieved_docs)} 条相关资料，来源: {', '.join(sources)}")
        print(f"\n🤖 ERNIE-4.5 正在思考中")
        
        # 2. 流式生成答案
        response_stream = self.generate_answer_stream(question, retrieved_docs)
        
        if response_stream is None:
            print("❌ 生成答案时遇到错误，请检查ERNIE模型服务是否正常。")
            return
        
        print("✨ 回答: ", end="", flush=True)
        
        # 处理流式响应
        full_answer = ""
        try:
            for chunk in response_stream:
                if chunk.choices[0].delta and chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    full_answer += content
            
            print(f"\n\n📖 参考来源: {', '.join(sources)}")
            print("-" * 60)
            
        except Exception as e:
            print(f"\n❌ 流式输出过程中出现错误: {e}")
            
        return full_answer



def interactive_rag_chat(rag_system: ERNIERAGSystem):
    """交互式RAG流式对话"""
    print("\n🎯 RAG智能问答系统 - 流式对话模式")
    print("✨ 基于本地ERNIE-4.5-21B-A3B模型")
    print("💡 输入问题开始对话，输入 'quit' 或 'exit' 退出")
    print("💡 输入 'stats' 查看知识库统计信息")
    print("💡 输入 'clear' 清屏")
    print("=" * 60)
    
    while True:
        try:
            question = input("\n❓ 请输入您的问题: ").strip()
            
            if question.lower() in ['quit', 'exit', '退出', 'q']:
                print("\n👋 感谢使用RAG智能问答系统，再见！")
                break
            elif question.lower() == 'stats':
                stats = rag_system.knowledge_base.get_collection_stats()
                print("\n📊 知识库统计信息:")
                total_docs = 0
                for name, info in stats.items():
                    count = info['document_count']
                    total_docs += count
                    if count > 0:
                        print(f"  ✅ {name}: {count} 个文档")
                    else:
                        print(f"  ⚪ {name}: {count} 个文档")
                print(f"📈 总计: {total_docs} 个文档")
                continue
            elif question.lower() == 'clear':
                import os
                os.system('cls' if os.name == 'nt' else 'clear')
                print("🎯 RAG智能问答系统 - 流式对话模式")
                continue
            elif not question:
                print("⚠️ 请输入有效的问题")
                continue
            
            # 进行RAG流式问答
            rag_system.ask_stream(question, top_k=3)
            
        except KeyboardInterrupt:
            print("\n\n👋 程序已中断，再见！")
            break
        except Exception as e:
            print(f"\n❌ 发生错误: {e}")
            print("💡 请检查ERNIE模型服务是否正常运行")

def main():
    """主程序入口"""
    print("🚀 初始化RAG智能问答系统...")
    
    try:
        # 直接初始化RAG系统
        rag_system = ERNIERAGSystem(
            ernie_host="0.0.0.0",
            ernie_port="7000",
            chroma_db_dir="./chroma_db",
            embedding_model="default"
        )
        
        # 检查知识库是否有数据
        stats = rag_system.knowledge_base.get_collection_stats()
        total_docs = sum(info['document_count'] for info in stats.values())
        
        if total_docs == 0:
            print("⚠️ 警告: 知识库中没有数据！")
            print("💡 请先运行以下命令构建知识库:")
            print("   python document_processor.py")
            print("   python chroma_builder.py")
            return
        
        print(f"✅ 知识库已加载，共 {total_docs} 个文档")
        
        # 启动交互式对话
        interactive_rag_chat(rag_system)
        
    except Exception as e:
        print(f"❌ 系统初始化失败: {e}")
        print("💡 请检查:")
        print("  1. ERNIE模型服务是否在7000端口运行")
        print("  2. 知识库目录 ./chroma_db 是否存在")
        print("  3. 相关依赖是否正确安装")

if __name__ == "__main__":
    main()

Building prefix dict from the default dictionary ...
2025-07-11 20:53:35,694 - jieba - DEBUG - Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2025-07-11 20:53:35,695 - jieba - DEBUG - Loading model from cache /tmp/jieba.cache
Loading model cost 0.677 seconds.
2025-07-11 20:53:36,372 - jieba - DEBUG - Loading model cost 0.677 seconds.
Prefix dict has been built successfully.
2025-07-11 20:53:36,376 - jieba - DEBUG - Prefix dict has been built successfully.
2025-07-11 20:53:36,952 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-07-11 20:53:37,009 - ChromaBuilder - INFO - 使用ChromaDB默认嵌入函数
2025-07-11 20:53:37,012 - ChromaBuilder - INFO - Chroma知识库初始化完成，数据目录: ./chroma_db


🚀 初始化RAG智能问答系统...
✅ 知识库已加载，共 9 个文档

🎯 RAG智能问答系统 - 流式对话模式
✨ 基于本地ERNIE-4.5-21B-A3B模型
💡 输入问题开始对话，输入 'quit' 或 'exit' 退出
💡 输入 'stats' 查看知识库统计信息
💡 输入 'clear' 清屏



❓ 请输入您的问题:  狼人



🔍 正在搜索相关资料...


2025-07-11 20:53:52,587 - ChromaBuilder - INFO - 查询 '狼人...' 返回 3 个结果
2025-07-11 20:53:52,599 - httpx - INFO - HTTP Request: POST http://0.0.0.0:7000/v1/chat/completions "HTTP/1.1 200 OK"


📚 找到 3 条相关资料，来源: knowledge_data/猎人玩法.txt, knowledge_data/狼人玩法.txt

🤖 ERNIE-4.5 正在思考中
✨ 回答: 根据提供的参考资料，以下是关于狼人玩法的详细回答：

### 狼人玩法的核心策略与技巧

1. **轮次压力应对表**  
   - **3狼**：激进悍跳，快速压缩好人信息空间，通过频繁发言或误导性信息混淆好人阵营。  
   - **2狼**：1狼倒钩+1狼深水，倒钩狼通过伪装好人归票关键神职，深水狼隐藏身份等待时机。  
   - **1狼**：彻底隐藏，利用好人逻辑漏洞归票关键神职（如预言家、女巫），避免过早暴露。  
   *信息来源：参考资料3（狼人玩法.txt）*

2. **高阶思维模型**  
   - **逆向思维推演**：假设自己是好人，构建逻辑链确保发言无矛盾，预判真预言家查验路径并干扰。  
   - **贝叶斯概率计算**：根据存活人数动态估算屠神/屠民成功率（如剩4民2神1狼时，屠民需3刀，屠神需2刀）。  
   - **信息污染技术**：  
     - **虚假共边关系**：故意保一个好人（如“X号绝对好人，出他我买单”），诱导女巫怀疑X。  
     - **时间线篡改**：伪造夜间信息（如“如果我是女巫，昨晚会毒Y号”），暗示Y可能有毒威胁。  
   *信息来源：参考资料3（狼人玩法.txt）*

3. **禁忌与容错机制**  
   - **绝对禁忌行为**：  
     - 夜间刀法违反优先级（如残局刀民却漏刀猎人）。  
     - 悍跳预言家报假查杀却选中真预言家。  
     - 倒钩狼过早暴露导致团队崩盘。  
   - **容错修复方案**：  
     - **刀中守卫盾**：次日谎称女巫用药，转移视线。  
     - **悍跳被识破**：其他狼人立刻倒钩，反咬其“狼查杀”。  
     - **误杀关键神职**：伪造遗言逻辑链，坐实其“自刀狼”身份。  
   *信息来源：参考资料3（狼人玩法.txt）*

4. **实战案例库**  
   - **自刀+倒钩组合拳**：  
     - 狼A首夜自刀，女巫救药→白天狼B悍跳发狼A金水→真预言家查验狼A为好人→狼C倒钩真预言家，最终污真预言家为狼。  
 


❓ 请输入您的问题:  平民玩法



🔍 正在搜索相关资料...


2025-07-11 20:54:41,438 - ChromaBuilder - INFO - 查询 '平民玩法...' 返回 3 个结果
2025-07-11 20:54:41,447 - httpx - INFO - HTTP Request: POST http://0.0.0.0:7000/v1/chat/completions "HTTP/1.1 200 OK"


📚 找到 3 条相关资料，来源: knowledge_data/猎人玩法.txt, knowledge_data/平民玩法.txt, knowledge_data/狼人玩法.txt

🤖 ERNIE-4.5 正在思考中
✨ 回答: 根据提供的参考资料，平民玩法的核心目标是通过信息整合与逻辑建模，构建多维视角分析网，辅助神职精准归狼，同时避免成为轮次消耗品。然而，参考资料中并未直接提供平民玩法的具体操作策略或详细步骤。

在狼人杀游戏中，平民（普通玩家）通常不拥有特殊技能，但可以通过观察其他玩家的行为、发言和投票模式来辅助神职玩家（如预言家、女巫、猎人等）进行归谬。以下是一些平民玩家可能采取的一般性策略：

1. **观察与记录**：平民玩家应密切关注其他玩家的发言和行为，记录下任何可疑或异常的举动。这有助于在后续的投票中做出更准确的判断。

2. **辅助神职**：平民玩家可以尝试与神职玩家建立联系，提供信息或支持他们的判断。例如，如果预言家查杀了某个玩家，平民玩家可以询问预言家的理由，并尝试验证这些信息的真实性。

3. **避免暴露**：平民玩家应尽量避免在游戏中暴露自己的身份或立场，以免成为狼队的攻击目标。这包括在发言中保持中立、不发表过于激进的言论等。

4. **理性投票**：在投票环节，平民玩家应基于观察到的信息和逻辑推理进行投票，避免盲目跟风或冲动投票。

5. **团队合作**：平民玩家应与其他好人玩家保持紧密合作，共同对抗狼队。这包括在游戏中分享信息、互相支持等。

需要注意的是，以上策略是基于狼人杀游戏的一般性规则和逻辑推理得出的，并非针对平民玩法的特定操作指南。由于参考资料中未提供平民玩法的具体策略，因此以上回答仅为一般性建议。

信息来源：平民玩法知识库（高阶分析版），知识库中强调了平民玩法的核心目标是通过信息整合与逻辑建模辅助神职归狼，但未提供具体操作策略。

📖 参考来源: knowledge_data/猎人玩法.txt, knowledge_data/平民玩法.txt, knowledge_data/狼人玩法.txt
------------------------------------------------------------


## 六、RAG的行业应用场景

### 6.1 企业客服
- 用RAG接入产品手册、售后流程，客服机器人实时调用最新资料回答用户问题，减少人工干预
- **本地化优势**：企业数据不出公司，保证数据安全

### 6.2 医疗辅助
- 医生输入患者症状，RAG检索最新临床指南和病例，辅助医生判断可能病因（需专业审核）
- **本地化优势**：患者隐私数据完全本地化处理

### 6.3 教育辅导
- 学生提问数学题，RAG从教材和习题集中找相关知识点，大模型基于教材内容讲解，确保和教学同步
- **本地化优势**：教学内容可控，无需担心网络连接问题

### 6.4 法律检索
- 律师输入案件情况，RAG检索相关法条和判例，大模型生成法律分析，提高检索效率
- **本地化优势**：案件信息保密，符合法律行业数据安全要求

## 七、初学者优化小技巧

1. **文本分块优化**：长文档拆分时，按"语义完整"拆分（如按段落），避免切断句子
   ```python
   from langchain.text_splitter import RecursiveCharacterTextSplitter
   text_splitter = RecursiveCharacterTextSplitter(
       chunk_size=500, 
       chunk_overlap=50,
       separators=["\n\n", "\n", "。", "！", "？", "；"]
   )
   chunks = text_splitter.split_text(long_document)
   ```

2. **提示词优化**：明确告诉模型"只基于提供的资料回答"
   ```
   请严格根据以下资料回答，资料外的信息不要提及。如果资料不足，直接说"根据提供的资料无法回答此问题"。
   资料：{context}
   问题：{question}
   ```

通过以上步骤，你已经掌握了基于本地部署ERNIE-4.5-21B-A3B模型的RAG系统核心逻辑和完整实现。这套方案既保证了数据安全（完全本地化），又提供了强大的知识问答能力，是企业级RAG应用的理想选择～